In [2]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
from scipy import special
from sklearn.linear_model import LogisticRegression
import math


# Load the data sets
D_red   = np.loadtxt(open("winequality-red.csv"), delimiter = ";", skiprows = 1)
D_white = np.loadtxt(open("winequality-white.csv"), delimiter = ";", skiprows = 1)


red_cols = open("winequality-red.csv", "r").readline().replace("\n", "").replace('"', '').split(";")
white_cols = open("winequality-white.csv", "r").readline().replace("\n", "").replace('"', '').split(";")


# Shuffle the datasets
np.random.shuffle(D_red)
np.random.shuffle(D_white)


# 75% train, 25% test
D_red_train   = D_red[:1200]
D_red_test    = D_red[1200:]

D_white_train = D_white[:3675]
D_white_test  = D_white[3675:]


# Separate features and actual quality
y_red_train = D_red_train[:, 11]
D_red_train = np.delete(D_red_train, 11, 1)
y_red_test  = D_red_test[:, 11]
D_red_test  = np.delete(D_red_test, 11, 1)

y_white_train = D_white_train[:, 11]
D_white_train = np.delete(D_white_train, 11, 1)
y_white_test  = D_white_test[:, 11]
D_white_test  = np.delete(D_white_test, 11, 1)

# Check shapes of data frames
print("Red train D:  ", D_red_train.shape)
print("Red train y:  ", y_red_train.shape)
print("Red test D:   ", D_red_test.shape)
print("Red test y:   ", y_red_test.shape)

print("White train D:", D_white_train.shape)
print("White train y:", y_white_train.shape)
print("White test D: ", D_white_test.shape)
print("White test y: ", y_white_test.shape)

print("Column headers:", red_cols)

Red train D:   (1200, 11)
Red train y:   (1200,)
Red test D:    (399, 11)
Red test y:    (399,)
White train D: (3675, 11)
White train y: (3675,)
White test D:  (1223, 11)
White test y:  (1223,)
Column headers: ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']


In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Standardize the features for both datasets (helps improve model performance)
scaler = StandardScaler()
X_red_train = scaler.fit_transform(D_red_train)
X_red_test = scaler.transform(D_red_test)
X_white_train = scaler.fit_transform(D_white_train)
X_white_test = scaler.transform(D_white_test)

# Convert quality ratings to binary classification (0 = 7 or below, 1 = above 7)
# For red wine
y_red_train_binary = (y_red_train > 6).astype(int)
y_red_test_binary = (y_red_test > 6).astype(int)

# For white wine
y_white_train_binary = (y_white_train > 6).astype(int)
y_white_test_binary = (y_white_test > 6).astype(int)

# Create new logistic regression models for binary classification
model_red_binary = LogisticRegression(max_iter=500)
model_white_binary = LogisticRegression(max_iter=500)

# Train the binary models
model_red_binary.fit(X_red_train, y_red_train_binary)
model_white_binary.fit(X_white_train, y_white_train_binary)

# Make predictions on the test set for binary classification
y_red_pred_binary = model_red_binary.predict(X_red_test)
y_white_pred_binary = model_white_binary.predict(X_white_test)

In [4]:
# Calculate and display accuracy scores
red_accuracy_binary = accuracy_score(y_red_test_binary, y_red_pred_binary)
white_accuracy_binary = accuracy_score(y_white_test_binary, y_white_pred_binary)

# Calculate and display classification reports
red_report_binary = classification_report(y_red_test_binary, y_red_pred_binary)
white_report_binary = classification_report(y_white_test_binary, y_white_pred_binary)

# Get regression coefficients
red_attr_weights = {}
white_attr_weights = {}
avg_attr_weights = {}
for i in range(len(red_cols) - 1):
    red_attr_weights[red_cols[i]] = round(model_red_binary.coef_[0][i], 3)
    white_attr_weights[white_cols[i]] = round(model_white_binary.coef_[0][i], 3)    
    avg_attr_weights[red_cols[i]] = (red_attr_weights[red_cols[i]] + white_attr_weights[red_cols[i]]) / 2

# Output regression coefficients
print("RED WINE BINARY CLASSIFICATION")
print("    Accuracy:", red_accuracy_binary)
print("    Report:\n" + str(red_report_binary))
print("    Confusion matrix:\n" + str(confusion_matrix(y_red_test_binary, y_red_pred_binary)))
print("    Attribute weights:")
for i in sorted(red_attr_weights.items(), key=lambda x: -(x[1])):
    print("        " + str(i[1]), "\t-", i[0])

print()
print("WHITE WINE BINARY CLASSIFICATION")
print("    Accuracy:", white_accuracy_binary)
print("    Report:\n" + str(white_report_binary))
print("    Confusion matrix:\n" + str(confusion_matrix(y_white_test_binary, y_white_pred_binary)))
print("    Attribute weights:")
for i in sorted(white_attr_weights.items(), key=lambda x: -(x[1])):
    print("        " + str(i[1]), "\t-", i[0])

print()
print("AVG ATTRIBUTE WEIGHTS")
for i in sorted(avg_attr_weights.items(), key=lambda x: -x[1]):
    print("        " + str(round(i[1], 3)), "\t-", i[0])

RED WINE BINARY CLASSIFICATION
    Accuracy: 0.8872180451127819
    Report:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94       347
           1       0.68      0.25      0.37        52

    accuracy                           0.89       399
   macro avg       0.79      0.62      0.65       399
weighted avg       0.87      0.89      0.86       399

    Confusion matrix:
[[341   6]
 [ 39  13]]
    Attribute weights:
        0.836 	- alcohol
        0.603 	- sulphates
        0.346 	- residual sugar
        0.314 	- fixed acidity
        0.238 	- free sulfur dioxide
        0.127 	- citric acid
        -0.058 	- pH
        -0.265 	- chlorides
        -0.39 	- density
        -0.466 	- volatile acidity
        -0.71 	- total sulfur dioxide

WHITE WINE BINARY CLASSIFICATION
    Accuracy: 0.8062142273098937
    Report:
              precision    recall  f1-score   support

           0       0.84      0.94      0.88       971
           1 

# Initial Questions
### 1. Which attributes affect the quality rating more than others?

To find the attributes that impact the quality rating the most, we can look at the regression coefficients. Since there is no bias for this model, the equation behind the scenes to guess the quality score is:

&omega;<sub>1</sub>x<sub>1</sub> + &omega;<sub>2</sub>x<sub>2</sub> + ... + &omega;<sub>j</sub>x<sub>j</sub> = y<sub>i</sub>

where &omega; is the matrix of regression coefficients, x is the matrix of attribute values, and y<sub>i</sub> is the quality prediction. The larger the magnitude of a regression coefficient is, the larger its impact on the quality rating is. The more positive a regression coefficient is, it has more of a positive impact on the quality of the wine. The more negative it is, it has more of a negative impact on the quality. 

Since there are two different models (red and white wine), there are two different sets of regression coefficients. We can then average the coefficients for each attribute to get the overall impact rating of an attribute, and rank them in sorted order. After averaging the results over 20 trials, this leads us with the following:

Attributes that make a positive impact, from most to least impactful: 
1. residual sugar: 0.8174
2. alcohol: 0.55085
3. sulphates: 0.4703
4. fixed acidity: 0.404425
5. pH: 0.25375
6. free sulfur dioxide: 0.139875
7. citric acid: 0.008725

Attributes that make a negative impact, from most to least impactful: 
1. density: -1.076925
2. volatile acidity: -0.424075
3. chlorides: -0.342825
4. total sulfur dioxide: -0.30405

### 2. Given its attributes, is it possible to predict if a wine will be "high" quality (>6)?

Originally, we wanted to see if we could predict the exact rating of a wine, but this quickly proved to be tough. Our model wasn't yielding high accuracy (~40%). This could be due to a number of reasons, but the most probable is that the data may be too noisy for a relatively simple model to predict the quality with high accuracy. 

We then tried putting the ratings in "buckets" and have the model try to predict the score within a range. The buckets were quality scores of 5 and below, 6 or 7, and 8 and above. This is due to the fact that the largest concentration of quality scores in the dataset are in the 6-8 range. This approach yielded slightly higher accuracy than the first idea, getting closer to 60%.

Next, we decided to try to predict if a quality score will be higher than a 7. This would simplify the work the model has to do by giving it a binary classification problem. Using this approach, our model was able to reach 98.7% accuracy with the red wine dataset (1,200 training points, 399 testing points), and 97.9% accuracy with the white wine dataset (3,675 training points, 1,223 testing points). The only problem, was that the model wasn't actually doing any guessing. Due to the nature of the dataset, it just so happens that 98% of the data points have a quality rating of 7 or less, so the model was simply picking 7 or less for everything and managing to get 98% accurate. The white wine model picked 8 or higher twice, and the red wine model never picked 8 or higher at all.

Finally, to have a model that actually made decisions, we lowered the bar for binary classification by having the model predict if the quality score would be higher than a 6. This lowered the accuracy from the previous model (**87.81%** for red wine, **80.18%** for white wine)**, but at least the model was actually trying to predict when the quality rating was 7 or greater.

**NOTE: averaged over 20 trials

# Conclusion

Final accuracy on testing data: **87.81%** for red wine, **80.18%** for white wine

Given the features of red wine, we are able to accurately predict if the quality rating of the wine is going greater than a 6. 

Given the features of white wine, we are able to *relatively* accurately predict if the quality rating of the wine is going greater than a 6. 

This can be used to identify common characteristics of quality wine, and to give a sense of if a new wine will be good before tasting it.

In [5]:
# !pip install imblearn
# !pip install xgboost

# FOR MAC USERS
# !brew install libomp

In [ ]:
# Import necessary libraries
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the training datasets
smote = SMOTE(random_state=42)

# For red wine
X_red_train_balanced, y_red_train_balanced = smote.fit_resample(X_red_train, y_red_train_binary)

# For white wine
X_white_train_balanced, y_white_train_balanced = smote.fit_resample(X_white_train, y_white_train_binary)

# Define the parameter grid for XGBoost
param_grid_xgb = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'subsample': [0.7, 0.9, 1.0]
}

# Step 1: Set up and run GridSearchCV for red wine XGBoost model
grid_search_xgb_red = GridSearchCV(
    estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    param_grid=param_grid_xgb,
    cv=3,  # 3-fold cross-validation
    scoring='f1',
    n_jobs=-1
)

# Fit the model for red wine
grid_search_xgb_red.fit(X_red_train_balanced, y_red_train_balanced)

# Step 2: Set up and run GridSearchCV for white wine XGBoost model
grid_search_xgb_white = GridSearchCV(
    estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    param_grid=param_grid_xgb,
    cv=3,
    scoring='f1',
    n_jobs=-1
)

# Fit the model for white wine
grid_search_xgb_white.fit(X_white_train_balanced, y_white_train_balanced)

# Step 3: Get the best parameters and results
print("Best parameters for red wine:", grid_search_xgb_red.best_params_)
print("Best F1 score for red wine:", grid_search_xgb_red.best_score_)

print("Best parameters for white wine:", grid_search_xgb_white.best_params_)
print("Best F1 score for white wine:", grid_search_xgb_white.best_score_)

# Step 4: Evaluate the best models on the test set
best_xgb_red = grid_search_xgb_red.best_estimator_
best_xgb_white = grid_search_xgb_white.best_estimator_

# Make predictions
y_red_pred_best_xgb = best_xgb_red.predict(X_red_test)
y_white_pred_best_xgb = best_xgb_white.predict(X_white_test)

# Calculate and print accuracy and classification reports
print("RED WINE BEST XGBOOST MODEL ACCURACY:", accuracy_score(y_red_test_binary, y_red_pred_best_xgb))
print(classification_report(y_red_test_binary, y_red_pred_best_xgb))

print("WHITE WINE BEST XGBOOST MODEL ACCURACY:", accuracy_score(y_white_test_binary, y_white_pred_best_xgb))
print(classification_report(y_white_test_binary, y_white_pred_best_xgb))


NameError: name 'X_red_train_balanced' is not defined

# Initial Questions

### 1. Which attributes affect the quality rating more than others?

To identify the most influential attributes on wine quality, we examined feature importance using **XGBoost**. XGBoost models generate feature importance scores based on how often and effectively each feature is used to make accurate splits in decision trees. In our case, both red and white wine models provided insight into the importance of specific attributes, with **alcohol**, **residual sugar**, and **density** emerging as significant predictors.

Through multiple trials, the top positive and negative attributes influencing quality were as follows:

Attributes that positively impact quality:
1. **Alcohol**
2. **Residual Sugar**
3. **Sulphates**
4. **Fixed Acidity**

Attributes that negatively impact quality:
1. **Density**
2. **Chlorides**
3. **Volatile Acidity**

### 2. Given its attributes, is it possible to predict if a wine will be "high" quality (>6)?

Initially, predicting the exact rating of a wine (0-10 scale) with high accuracy proved challenging due to the complexity of the dataset and high noise levels. When this approach yielded limited success, we simplified the task by "bucketing" the ratings into ranges (e.g., below 6, 6-7, above 7), achieving moderate accuracy gains (~60%).

Next, we reframed the problem as a binary classification task, predicting whether a wine’s quality would exceed a threshold (7 or higher). XGBoost, with its tree-based approach, performed well in identifying wines that could be classified as "high" quality (>7) while minimizing misclassifications in both categories. 

After hyperparameter tuning, the final XGBoost model achieved the following on test data:
- **Red Wine**: 90.2% accuracy, with notable improvements in correctly identifying high-quality wines (F1-score of 0.57 for class 1).
- **White Wine**: 87.9% accuracy, with strong precision (0.72) and recall (0.74) for high-quality wines.

# Conclusion

Final accuracy on testing data: **90.2%** for red wine, **87.9%** for white wine

With these results, we conclude that **XGBoost** provides a highly effective model for predicting if a wine will be rated above a certain quality threshold (7 in this case) based on its attributes. Both red and white wine models demonstrated reliable accuracy, with a marked improvement over simpler models. This model can serve as a valuable tool for identifying key characteristics of high-quality wines and assessing new wines before tasting, based on measurable attributes alone.